In [ ]:
import xml.etree.ElementTree as ET
import numpy as np
import pandas as pd
import subprocess

Inicialmente é necessário criar uma série de funções para retirada dos dados necessários do gbXML para criação do input para o metamodelo. As transformações geométricas (principalmente retirada de áreas a partir de coordenadas de pontos) são realizadas durante essa etapa.
A partir dos dados levantados e manipulados é criado o dataframe de entrada de dados para o metamodelo.

Função para calcular a área de qualquer polígono a partir de coordenadas xyz.

In [ ]:
#vetor unitário normal da superfície definida pelos pontos a, b e c
def unit_normal(a, b, c):
    x = np.linalg.det([[1,a[1],a[2]],
         [1,b[1],b[2]],
         [1,c[1],c[2]]])
    y = np.linalg.det([[a[0],1,a[2]],
         [b[0],1,b[2]],
         [c[0],1,c[2]]])
    z = np.linalg.det([[a[0],a[1],1],
         [b[0],b[1],1],
         [c[0],c[1],1]])
    magnitude = (x**2 + y**2 + z**2)**.5
    return (x/magnitude, y/magnitude, z/magnitude)

#área do polígono 'poly'
def poly_area(poly):
    if len(poly) < 3:
        return 0
    total = [0, 0, 0]
    N = len(poly)
    for i in range(N):
        vi1 = poly[i]
        vi2 = poly[(i+1) % N]
        prod = np.cross(vi1, vi2)
        total[0] += prod[0]
        total[1] += prod[1]
        total[2] += prod[2]
    result = np.dot(total, unit_normal(poly[0], poly[1], poly[2]))
    return abs(result/2)

#https://stackoverflow.com/questions/12642256/python-find-area-of-polygon-from-xyz-coordinates

Função para extrair as propriedades de superfícies do arquivo gbXML e devolvê-las em formato de lista (nome da superfície, nome do espaço que a superfície está vinculada, sua área e seu azimute).

In [ ]:
def surface_area(surfaces):
    
    surface_properties = []
    
    for i in range(len(surfaces)):
        surface_name = surfaces[i].find('{http://www.gbxml.org/schema}Name').text
        surface_space = surfaces[i].find('{http://www.gbxml.org/schema}AdjacentSpaceId').attrib['spaceIdRef']
        surface_azimuth = float(surfaces[i].find('.//{http://www.gbxml.org/schema}Azimuth').text)
        planargeometry = surfaces[i].find('.//{http://www.gbxml.org/schema}PlanarGeometry')
        surface_points = planargeometry.findall('.//{http://www.gbxml.org/schema}CartesianPoint')

        surface_coord = []

        for j in range(len(surface_points)): 
            surface_points_xyz = []
            for k in range(3):
                surface_points_xyz.append(float(surface_points[j][k].text))
            surface_coord.append(surface_points_xyz)
            
        surface_area = poly_area(surface_coord)
        
        surface_properties.append([surface_name, 
                                   surface_space, 
                                   surface_area,
                                  surface_azimuth])
        
    return (surface_properties)

Função para extrair as informações de aberturas (apenas janelas operáveis) do arquivo gbXML e devolvê-las em formato de lista (nome da superfície na qual a abertura está inserida, nome do espaço que a superfície pai está vinculada, área da abertura e azimute da superfície pai).

In [ ]:
def opening_area(surfaces):
    
    surface_properties = []
    
    for i in range(len(surfaces)):
        surface_name = surfaces[i].find('{http://www.gbxml.org/schema}Name').text
        surface_space = surfaces[i].find('{http://www.gbxml.org/schema}AdjacentSpaceId').attrib['spaceIdRef']
        surface_azimuth = float(surfaces[i].find('.//{http://www.gbxml.org/schema}Azimuth').text)
        openings = surfaces[i].findall('.//{http://www.gbxml.org/schema}Opening[@openingType="OperableWindow"]')
        for j in range(len(openings)):
            planargeometry = openings[j].find('.//{http://www.gbxml.org/schema}PlanarGeometry')
            opening_points = planargeometry.findall('.//{http://www.gbxml.org/schema}CartesianPoint')

            surface_coord = []

            for k in range(len(opening_points)): 
                opening_points_xyz = []
                for l in range(3):
                    opening_points_xyz.append(float(opening_points[k][l].text))
                surface_coord.append(opening_points_xyz)
            
            surface_area = poly_area(surface_coord)
        
            surface_properties.append([surface_name, 
                                   surface_space, 
                                   surface_area,
                                      surface_azimuth])
        
    return (surface_properties)

Função para realizar o somatório de áreas das superfícies por orientação baseado no dado de azimute e devolver a informação no formato de lista (norte, leste, sul, oeste).

In [ ]:
def surface_area_orientation(surfaces):

    surfaces_north = [0]
    surfaces_east = [0]
    surfaces_west = [0]
    surfaces_south = [0]

    for i in range(len(surfaces)):
    
        if surfaces[i][3]<=45 and surfaces[i][3]>=0 or surfaces[i][3]<=360 and surfaces[i][3]>315:
            surfaces_north.append(surfaces[i][2])
        elif surfaces[i][3]<=135 and surfaces[i][3]>45:
            surfaces_east.append(surfaces[i][2])
        elif surfaces[i][3]<=315 and surfaces[i][3]>225:
            surfaces_west.append(surfaces[i][2])    
        elif surfaces[i][3]<=225 and surfaces[i][3]>135:
            surfaces_south.append(surfaces[i][2])   
        
    surfaces_north = np.array(surfaces_north).sum()
    surfaces_east = np.array(surfaces_east).sum()
    surfaces_west = np.array(surfaces_west).sum()
    surfaces_south = np.array(surfaces_south).sum()
    
    return [surfaces_north, surfaces_east, surfaces_south, surfaces_west]

Funções para realizar o cálculo do WWR dividindo a área envidraçada pela área de parede, retornando o valor de zero para divisões por zero.

In [ ]:
def safe_div(x,y):
    if y==0: return 0
    return x/y

def wwr_calculator(opening_area,extwall_area):
    wwr=[]
    for i in range(4):
        wwr.append(safe_div(opening_area[i],extwall_area[i]))
    return wwr    

Função para definir a condição de exposição das superfícies horizontais (piso, cobertura, pilotis) realizando o somatório de superfícies em determinada condição no espaço e comparando com a área do espaço em questão. Se a área total da superfície em determinada condição superar 50% da área do espaço, essa condição será a determinante para a entrada do metamodelo.

In [ ]:
def exposition(surfaces_area,space_id,space_area):    
    
    surfaces_area_space = []
    
    for j in range(len(surfaces_area)):
        if surfaces_area[j][1]==space_id:
            surfaces_area_space.append(surfaces_area[j][2])

    surfaces_area_space = sum(surfaces_area_space)
    
    if surfaces_area_space >= 0.5*space_area:
        exp = 1
    else:
        exp = 0
        
    return exp

Algoritmo para carregar o arquivo gbXML e começar a extração das informações indo até o nível dos espaços e pavimentos do modelo.

In [ ]:
tree = ET.parse('MCMV_Unifamiliar_Revit.xml')
print('loading complete')
root = tree.getroot()
campus = root.find('{http://www.gbxml.org/schema}Campus')
building = campus.find('{http://www.gbxml.org/schema}Building')
spaces = building.findall('{http://www.gbxml.org/schema}Space[@conditionType="HeatedAndCooled"]')
buildingstorey = building.findall('{http://www.gbxml.org/schema}BuildingStorey')

Retirada de todas as superfícies de interesse do arquivo XML (paredes externas, pisos em contato com o solo, pilotis, pisos intermediários, coberturas e aberturas).

In [ ]:
# Find all the external walls, openings, roof, slabs on grade and floor in the campus object
surfaces = campus.findall("{http://www.gbxml.org/schema}Surface")
surfaces_extwall = campus.findall("{http://www.gbxml.org/schema}Surface[@surfaceType='ExteriorWall']")
surfaces_slab = campus.findall("{http://www.gbxml.org/schema}Surface[@surfaceType='SlabOnGrade']")
surfaces_roof = campus.findall("{http://www.gbxml.org/schema}Surface[@surfaceType='Roof']")
surfaces_floor = campus.findall("{http://www.gbxml.org/schema}Surface[@surfaceType='InteriorFloor']")
surfaces_raisedfloor = campus.findall("{http://www.gbxml.org/schema}Surface[@surfaceType='RaisedFloor']")
surfaces_openings = campus.findall('.//{http://www.gbxml.org/schema}Opening[@openingType="OperableWindow"]')

Realização do cálculo de área de cada superfície retornando uma lista contendo as informações de nome da superfície, nome do espaço que a superfície está vinculada, sua área e seu azimute.

In [ ]:
extwall_area = surface_area(surfaces_extwall)
opening_area = opening_area(surfaces_extwall)
slabs_area = surface_area(surfaces_slab)
interiorfloors_area = surface_area(surfaces_floor)
roofs_area = surface_area(surfaces_roof)
raisedfloors_area = surface_area(surfaces_raisedfloor)

Criação do dataframe para organização dos dados de entrada necessários ao metamodelo.

In [ ]:
df = pd.DataFrame(np.nan, index=range(len(spaces)), 
                  columns=['space_storey_name','space_storey_id','space_name','space_id','zona',
                           'wwr_norte','wwr_leste','wwr_sul','wwr_oeste',
                           'area_par_exp_norte','area_par_exp_leste',
                           'area_par_exp_sul','area_par_exp_oeste',
                           'area_zona','ct_par_ext','u_par_ext','ct_cob','u_cob',
                           'u_vid','fs_vid','tipo_pav','pedireito','abspar','abscob',
                           'tamanhoprojecao','hpav','veneziana','hjan','openfac',
                           'pilotis','exp_pis','exp_cob',
                           'TMA','dpT','AMA','dpA'])

Algoritmo para buscar as informações de 'id', nome e valor de altura do pavimento para cada um dos níveis encontrados no arquivo gbXML.

In [ ]:
building_storey = []
for i in range(len(buildingstorey)):
    building_storey_id = buildingstorey[i].attrib['id']
    building_storey_name = buildingstorey[i].find('{http://www.gbxml.org/schema}Name').text
    building_storey_level = float(buildingstorey[i].find('{http://www.gbxml.org/schema}Level').text)
    
    building_storey.append([building_storey_id,building_storey_name,building_storey_level])

Algoritmo para organização e preenchimento do dataframe por zona térmica encontrada no metamodelo, realizando primeiramente a divisão das superfícies por espaço adjacente e depois por orientação em relação ao norte.

In [ ]:
for i in range(len(spaces)):
    space_id = spaces[i].attrib['id']    
    space_name = spaces[i].find('{http://www.gbxml.org/schema}Name').text
    space_storey_id = spaces[i].attrib['buildingStoreyIdRef']
    
    for j in range(len(building_storey)):
        if building_storey[j][0]==space_storey_id:
            space_storey_name = building_storey[j][1]
            space_storey_level = building_storey[j][2]
    
    space_area = float(spaces[i].find('{http://www.gbxml.org/schema}Area').text)
    space_volume = float(spaces[i].find('{http://www.gbxml.org/schema}Volume').text)
    space_height = space_volume/space_area
    
    extwall_area_space = []
    
    for j in range(len(extwall_area)):
        if extwall_area[j][1]==space_id:
            extwall_area_space.append(extwall_area[j])
    
    opening_area_space = []
    
    for j in range(len(opening_area)):
        if opening_area[j][1]==space_id:
            opening_area_space.append(opening_area[j])
    
    extwall_area_orientation = surface_area_orientation(extwall_area_space)
    opening_area_orientation = surface_area_orientation(opening_area_space)

    wwr = wwr_calculator(opening_area_orientation,extwall_area_orientation)
    
    exp_pis = exposition(slabs_area,space_id,space_area)
    exp_cob = exposition(roofs_area,space_id,space_area)     
    pilotis = exposition(raisedfloors_area,space_id,space_area)
    
    if "sala" in space_name.lower():
        zona = 0
    elif "quarto" in space_name.lower():
        zona = 1
    else:
        zona = 2
    
    ct_par_ext = 2 #kJ/m².K
    u_par_ext = 3.65 #W/m².K
    abspar = 0.6

    ct_cob = 1 #kJ/m².K
    u_cob = 2.02 #W/m².K
    abscob = 0.6

    u_vid = 5.7  #W/m².K
    fs_vid = 0.87

    openfac = 0.5
    hjan = 1/space_height
    veneziana = 0
    tamanhoprojecao = 0

    tipo_pav = 0
    
    TMA = 21.47
    dpT = 3.01
    AMA = 5.09
    dpA = 1.07
    
    df.iloc[i] = [space_storey_name,space_storey_id,space_name,space_id,zona,
                  wwr[0],wwr[1],wwr[2],wwr[3],
                  extwall_area_orientation[0],extwall_area_orientation[1],
                  extwall_area_orientation[2],extwall_area_orientation[3],
                  space_area,ct_par_ext,u_par_ext,ct_cob,u_cob,u_vid,fs_vid,tipo_pav,
                  space_height,abspar,abscob,tamanhoprojecao,space_storey_level,
                  veneziana,hjan,openfac,pilotis,exp_pis,exp_cob,
                  TMA,dpT,AMA,dpA]
df

Transformação do dataframe em arquivo 'csv' para servir de input para a função em R acionar a rede neural.

In [ ]:
df.to_csv('inputs.csv',encoding='utf-8-sig')

Código que aciona a função 'prediction.R' e espera a função ser realizada para seguir com o código adiante.

In [ ]:
subprocess.check_call(['Rscript', 'prediction.R'], shell=False)

Função para realizar a leitura do arquivo 'csv' com os resultados de carga térmica de aquecimento e resfriamento por zona térmica considerada previstos pelo metamodelo.

In [ ]:
dfcompleto = pd.read_csv('resultados.csv')
dfcompleto

Montagem de um dataframe intermediário para realizar o cálculo da carga térmica total da edificação em cada um dos casos a ser realizado.

In [ ]:
dfedificio = pd.DataFrame(np.nan, index=range(1), 
                  columns=['ct_par_ext','u_par_ext','ct_cob','u_cob',
                           'u_vid','fs_vid','tipo_pav','abspar','abscob',
                           'tamanhoprojecao','veneziana','hjan','openfac',
                           'heating','cooling'])

Função para criação do dataframe final com as características definidoras de cada caso e seu resultado de carga térmica de aquecimento e resfriamento total.

In [ ]:
df = pd.DataFrame({})
for i in range(1):
    dfedificio=(dfcompleto.loc[[i],['ct_par_ext','u_par_ext','ct_cob','u_cob',
                             'u_vid','fs_vid','tipo_pav','abspar','abscob',
                             'tamanhoprojecao','veneziana','hjan','openfac']])
    
    dfresultado=(dfcompleto.loc[0:(len(spaces)),['heating','cooling']])   
    dfresultadototal=pd.DataFrame({'total heating': [dfresultado['heating'].sum()],'total cooling':[dfresultado['cooling'].sum()]})
    
    dftemporario = dfedificio.join(dfresultadototal)
    #join para juntar as colunas de resultado de carga térmica, depois append para adicionar a linha de cada caso
    df = pd.concat([df,dftemporario])
df